<a href="https://colab.research.google.com/github/xoxominji/22-1-ESAA-Practice/blob/main/0314_session_%EB%B0%95%EB%AF%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

핸즈온 3장 연습문제

1번

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame= False)

In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
import numpy as np
y = y.astype(np.uint8)

In [4]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [5]:
#knn 기본
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)

KNeighborsClassifier()

In [9]:
knn_pred = knn_clf.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, knn_pred)

0.9688

In [12]:
#하이퍼파라미터 튜닝
grid_params = {'weights': ['uniform', 'distance'],
               'n_neighbors': [3,4,5]
               }

In [15]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(knn_clf, grid_params, cv=3)
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 4, 5],
                         'weights': ['uniform', 'distance']})

In [17]:
gs.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [19]:
gs.best_score_

0.9703500000000002

In [20]:
y_pred = gs.predict(X_test)
accuracy_score(y_test, y_pred)

0.9714

2번 (데이터 증식, 훈련 세트 확장)

In [21]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    #boundary 외부는 "constant"한 값으로 채워넣고 그 값은 0으로 설정
    return shifted_image.reshape([-1])

In [24]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [27]:
shuffle_idx = np.random.permutation(len(X_train_augmented)) #무작위로 섞인 배열
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [28]:
knn_clf = KNeighborsClassifier(**gs.best_params_) #1번에서 만들어놓은 모델

In [29]:
knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [30]:
y_pred = knn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9763

> 데이터 증식을 통해 정확도가 0.5 올라감